In [ ]:
# Instalação das bibliotecas necessárias
!pip install -q nltk spacy gensim scikit-learn matplotlib
!python -m spacy download pt_core_news_sm


In [ ]:
# @title
import re
import string
import unicodedata
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
import spacy

# Downloads necessários do NLTK
nltk.download("stopwords")
nltk.download("rslp")

# Carregamento de recursos
nlp = spacy.load("pt_core_news_sm")
stopwords_pt = set(stopwords.words("portuguese"))
stemmer = RSLPStemmer()

# -----------------------------
# Base de dados simulada
# -----------------------------
dados = [
    ("Entrega muito rápida, adorei o produto!", "positivo"),
    ("Produto chegou quebrado e com defeito.", "negativo"),
    ("Demorou demais para chegar.", "negativo"),
    ("Atendimento excelente e muito educado.", "positivo"),
    ("Embalagem rasgada, mas produto ok.", "neutro"),
    ("Preço justo e entrega no prazo.", "positivo"),
    ("Suporte demorou para responder.", "negativo"),
    ("Qualidade ótima, recomendo.", "positivo"),
    ("Não funcionou corretamente.", "negativo"),
    ("Experiência normal, nada demais.", "neutro"),
]

df = pd.DataFrame(dados, columns=["texto", "sentimento"])

# -----------------------------
# Funções de limpeza
# -----------------------------
def limpar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r"http\S+", "", texto)
    texto = unicodedata.normalize("NFKD", texto)
    texto = "".join(c for c in texto if not unicodedata.combining(c))
    texto = texto.translate(str.maketrans("", "", string.punctuation))
    return texto

def preprocessar(texto):
    texto_limpo = limpar_texto(texto)
    tokens = texto_limpo.split()
    tokens = [t for t in tokens if t not in stopwords_pt and len(t) > 2]
    stems = [stemmer.stem(t) for t in tokens]
    lemas = [token.lemma_ for token in nlp(" ".join(tokens))]
    return texto_limpo, tokens, stems, lemas

# Aplicação do pré-processamento
df["texto_limpo"], df["tokens"], df["stems"], df["lemas"] = zip(
    *df["texto"].apply(preprocessar)
)

df.head()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from gensim.models import Word2Vec

# TF-IDF
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df["texto_limpo"])

# Redução de dimensionalidade
svd = TruncatedSVD(n_components=2, random_state=42)
X_svd = svd.fit_transform(X_tfidf)

# Word2Vec
w2v = Word2Vec(
    sentences=df["tokens"],
    vector_size=50,
    window=5,
    min_count=1,
    epochs=50
)

w2v.wv.most_similar("entrega")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Codificação dos rótulos
mapa = {"negativo": 0, "neutro": 1, "positivo": 2}
df["y"] = df["sentimento"].map(mapa)

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, df["y"], test_size=0.3, random_state=42
)

# Modelo
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_train, y_train)

# Avaliação
y_pred = modelo.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans

# -----------------------------
# LDA – Tópicos
# -----------------------------
count_vec = CountVectorizer()
X_count = count_vec.fit_transform(df["texto_limpo"])

lda = LatentDirichletAllocation(n_components=3, random_state=42)
lda.fit(X_count)

print("Tópicos identificados:")
for i, topic in enumerate(lda.components_):
    palavras = [count_vec.get_feature_names_out()[j] for j in topic.argsort()[-6:]]
    print(f"Tópico {i}: {', '.join(palavras)}")

# -----------------------------
# Clustering
# -----------------------------
kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)
df["cluster"] = kmeans.fit_predict(X_svd)

# -----------------------------
# Gráficos
# -----------------------------
plt.figure()
df["sentimento"].value_counts().plot(kind="bar", title="Distribuição de Sentimentos")
plt.show()

plt.figure()
for c in df["cluster"].unique():
    idx = df["cluster"] == c
    plt.scatter(X_svd[idx, 0], X_svd[idx, 1], label=f"Cluster {c}")
plt.legend()
plt.title("Clusters de Feedbacks")
plt.show()
